In [ ]:

!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
pip install pandas torch

In [ ]:
!pip install tqdm

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer,BertForSequenceClassification, TrainingArguments,Trainer

In [ ]:
from tqdm import tqdm

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
#device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Load the data
df=pd.read_csv('/content/drive/MyDrive/datasets/mental_health_data/Combined Data.csv')

In [ ]:
df.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [ ]:
df.columns

Index(['Unnamed: 0', 'statement', 'status'], dtype='object')

In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df.head(2)

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety


In [ ]:
df['status'].unique()

array(['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar',
       'Personality disorder'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
df['status']=le.fit_transform(df['status'])

In [ ]:
df.head()

,statement,status
0,oh my gosh,0
1,"trouble sleeping, confused mind, restless hear...",0
2,"All wrong, back off dear, forward doubt. Stay ...",0
3,I've shifted my focus to something else but I'...,0
4,"I'm restless and restless, it's been a month n...",0


In [ ]:
df['status'].value_counts()

,count
status,
3,16351
2,15404
6,10653
0,3888
1,2877
5,2669
4,1201


In [ ]:
import re

def clean_statement(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text





In [ ]:
df['statement'].isna().sum()

362

In [ ]:
df.dropna(subset='statement',inplace=True)

In [ ]:
# Apply the cleaning function to the 'statement' column
df['statement'] = df['statement'].apply(clean_statement)

In [ ]:
#convert dataset into HuggingFace dataset
dataset=Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['statement', 'status', '__index_level_0__'],
    num_rows: 52681
})

In [ ]:
dataset=dataset.train_test_split(test_size=0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['statement', 'status', '__index_level_0__'],
        num_rows: 42144
    })
    test: Dataset({
        features: ['statement', 'status', '__index_level_0__'],
        num_rows: 10537
    })
})

#Tokenization

In [ ]:
# Loading the BERT Tokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# tokenize function

def tokenize_function(examples):
  return tokenizer(examples['statement'],padding='max_length',truncation=True)

#Tokenize the datasets

tokenized_datasets=dataset.map(tokenize_function,batched=True)




Map:   0%|          | 0/42144 [00:00<?, ? examples/s]

Map:   0%|          | 0/10537 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['statement', 'status', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 42144
    })
    test: Dataset({
        features: ['statement', 'status', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10537
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.map(lambda x: {'labels': x['status']})

Map:   0%|          | 0/42144 [00:00<?, ? examples/s]

Map:   0%|          | 0/10537 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['statement','status','__index_level_0__','token_type_ids'])

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 42144
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10537
    })
})

In [ ]:
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

In [ ]:
#Load the BERT model for sequence classification
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=len(df['status'].unique())).to('cuda')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    save_strategy='epoch',
    save_steps=500,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda pred: {'accuracy': torch.sum(torch.tensor(pred.label_ids == pred.predictions.argmax(-1))) / len(pred.label_ids)},
)

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.432800,0.411780,0.841131


TrainOutput(global_step=2634, training_loss=0.555728360569287, metrics={'train_runtime': 4027.0256, 'train_samples_per_second': 10.465, 'train_steps_per_second': 0.654, 'total_flos': 1.10890501152768e+16, 'train_loss': 0.555728360569287, 'epoch': 1.0})

In [ ]:
model.save_pretrained('./saved_model')

In [ ]:
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.41177991032600403, 'eval_accuracy': 0.8411312699317932, 'eval_runtime': 292.9701, 'eval_samples_per_second': 35.966, 'eval_steps_per_second': 0.563, 'epoch': 1.0}
